In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split, GridSearchCV, PredefinedSplit
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, RobustScaler
from collections import Counter
from Model_Utils import Sequence_Model, Classify_Model, Embedding_Model, HeadProjection_Model, Model_Combine
from Dataset_Utils import *
from General_Utils import *
from Loss_Utils import *
from torch.optim.lr_scheduler import ReduceLROnPlateau
import joblib
from scipy import stats
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
#torch.autograd.set_detect_anomaly(True)

# 14, 6, 17, 3, 2, 13, 9, 10, 15, 8, 7, 11, 4, 5, 16
SUBJECT_ID_TEST = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
SUBJECT_ID_TEST = ['S'+str(x) for x in SUBJECT_ID_TEST]
# SUBJECT_ID_TEST = 'S16'
result_personal = {}

# # 'GM1', 'EK1', 'NM1', 'RY1', 'KSG1', 'AD1', 'NM3', 'SJ1', 'BK1', 'RY2', 'GM2', 'MT1', 'NM2'
# SUBJECT_ID_TEST = 'AD1' # SJ1

USE_LARS = True
OPTIM = 'Adam'
LR = 0.003

MARGIN = 1
SEQ_DIM = 128
INPUT_FT = 60
EMBEDDING_HIDDEN = [256, 512]
PROJECTION_OUT = None #128 # None or int
CLASSIFY_HIDDEN = [-1] # [64] # list or None
DROPOUT = 0.2

MAX_EPOCH = 50
LOOK_BEFORE = 3
INTERNAL_SAMPLE = 200

MODEL_NAME = 'Euclid_CLS_512_LARS_Adam'
SAVE_MODEL_DIR = 'Output'
NAME_DATASET = 'WESAD'

##### READ DATASET #####
if NAME_DATASET == 'WESAD':
    DATA_DIR = '/home/nvtu/PhD_Work/StressDetection/DATA/MyDataset/WESAD'
    data_group = np.load(f'{DATA_DIR}/{NAME_DATASET}_WRIST_groups_1_60.npy')
    data_gt = np.load(f'{DATA_DIR}/{NAME_DATASET}_WRIST_ground_truth_1_60.npy')
    data_ft = np.load(f'{DATA_DIR}/{NAME_DATASET}_WRIST_stats_feats_1_60.npy')
else:
    DATA_DIR = '/home/nvtu/PhD_Work/StressDetection/DATA/MyDataset/AffectiveROAD_Data/Database'
    NAME_DATASET = 'AffectiveROAD'
    data_group = np.load(f'{DATA_DIR}/{NAME_DATASET}_groups_1.npy')
    data_gt = np.load(f'{DATA_DIR}/{NAME_DATASET}_ground_truth_1.npy')
    data_ft = np.load(f'{DATA_DIR}/{NAME_DATASET}_stats_feats_1.npy')
    indices = np.where(data_gt >= 0)[0]
    data_ft = data_ft[indices]
    data_group = data_group[indices]
    data_gt = data_gt[indices]

# Create dataframe for dataset
column_values = [f'f{x}' for x in range(data_ft.shape[1])]
data_full = pd.DataFrame(data = data_ft,  
                         columns = column_values)
data_full['subject_id'] = data_group
data_full['label'] = data_gt
list_subject_id = np.unique(data_full['subject_id']).tolist()

##### TRAIN / VAL / TEST #####
for subject_id_test in SUBJECT_ID_TEST:
    print(f'Processing {subject_id_test}')
    data_test = data_full[data_full.subject_id == subject_id_test]
    ft_names = data_test.columns.tolist()

    # Scaler Data
    X_test = data_test.iloc[:,:-2].to_numpy()
    group_test = data_test.iloc[:,-2].to_numpy()
    y_test = data_test.iloc[:,-1].to_numpy()

    X_test = X_test.astype('float64')
    y_test = y_test.astype('float64')
    
    skf = StratifiedKFold(n_splits=3)
    list_fold = {}
    count = 0
    for train_index, test_index in skf.split(X_test, y_test):
        list_fold[count] = {}
        list_fold[count]['train_idx'] = train_index
        list_fold[count]['test_idx'] = test_index
        count += 1
    
    sum_bacc = 0
    for f in range(3):
        train_index = list_fold[f]['train_idx']
        test_index = list_fold[f]['test_idx']
        X_train = X_test[train_index]
        y_train = y_test[train_index]
        X_test_t = X_test[test_index]
        y_test_t = y_test[test_index]
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test_t = scaler.transform(X_test_t)
        clf = RandomForestClassifier(n_estimators = 1000, random_state = 0, 
                                     n_jobs = -1, max_features='sqrt', 
                                     oob_score=True, bootstrap=True, class_weight = 'balanced')
        #clf = SVC(kernel='rbf', random_state=0, 
        #          class_weight='balanced', C=10)
        clf.fit(X_train, y_train)
        # Prediction
        Y_pred_test = clf.predict(X_test_t)
        acc_test = accuracy_score(y_test_t, Y_pred_test)
        f1_test = f1_score(y_test_t, Y_pred_test)
        bacc_test = balanced_accuracy_score(y_test_t, Y_pred_test)
        sum_bacc += bacc_test
    sum_bacc /= 3
    result_personal[subject_id_test] = sum_bacc
    print(sum_bacc)    

Processing S2
0.9066982912115655
Processing S3
0.6659446350826342
Processing S4
1.0
Processing S5
0.9842501175364363
Processing S6
0.991654744873629
Processing S7
0.9861626412087617
Processing S8
0.9854554124940392
Processing S9
1.0
Processing S10
1.0
Processing S11
1.0
Processing S13
1.0
Processing S14
1.0
Processing S15
0.9823717948717948
Processing S16
1.0
Processing S17
0.9011021500342652


In [2]:
result_personal

{'S2': 0.9066982912115655,
 'S3': 0.6659446350826342,
 'S4': 1.0,
 'S5': 0.9842501175364363,
 'S6': 0.991654744873629,
 'S7': 0.9861626412087617,
 'S8': 0.9854554124940392,
 'S9': 1.0,
 'S10': 1.0,
 'S11': 1.0,
 'S13': 1.0,
 'S14': 1.0,
 'S15': 0.9823717948717948,
 'S16': 1.0,
 'S17': 0.9011021500342652}

In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split, GridSearchCV, PredefinedSplit
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, RobustScaler
from collections import Counter
from Model_Utils import Sequence_Model, Classify_Model, Embedding_Model, HeadProjection_Model, Model_Combine
from Dataset_Utils import *
from General_Utils import *
from Loss_Utils import *
from torch.optim.lr_scheduler import ReduceLROnPlateau
import joblib
from scipy import stats
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
#torch.autograd.set_detect_anomaly(True)

# 14, 6, 17, 3, 2, 13, 9, 10, 15, 8, 7, 11, 4, 5, 16
SUBJECT_ID_TEST = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
SUBJECT_ID_TEST = ['S'+str(x) for x in SUBJECT_ID_TEST]
SUBJECT_ID_TEST = 'S14'
subject_id_test = SUBJECT_ID_TEST
result_personal = {}

# # 'GM1', 'EK1', 'NM1', 'RY1', 'KSG1', 'AD1', 'NM3', 'SJ1', 'BK1', 'RY2', 'GM2', 'MT1', 'NM2'
# SUBJECT_ID_TEST = 'AD1' # SJ1

USE_LARS = True
OPTIM = 'Adam'
LR = 0.003

EDA_FT = 35
EMB_EDA_SIZE = [70, 35]
HR_FT = 25
EMB_HR_SIZE = [50, 25]
CLS_SIZE = None # None or List

MAX_EPOCH = 50
USE_RES = False
OPTIM = 'Adam'
LR = 0.003
DROPOUT = 0.4
ACTIVATION = 'relu'
MODEL_NAME = 'Adam_Encode_Balance_W60'
SAVE_MODEL_DIR = 'Output_CLS_Val_X'
SUBFOLDER = 'Encode_Balance_DO4'
NAME_DATASET = 'WESAD'

##### READ DATASET #####
if NAME_DATASET == 'WESAD':
    DATA_DIR = '/home/nvtu/PhD_Work/StressDetection/DATA/MyDataset/WESAD'
    data_group = np.load(f'{DATA_DIR}/{NAME_DATASET}_WRIST_groups_1_60.npy')
    data_gt = np.load(f'{DATA_DIR}/{NAME_DATASET}_WRIST_ground_truth_1_60.npy')
    data_ft = np.load(f'{DATA_DIR}/{NAME_DATASET}_WRIST_stats_feats_1_60.npy')
else:
    DATA_DIR = '/home/nvtu/PhD_Work/StressDetection/DATA/MyDataset/AffectiveROAD_Data/Database'
    NAME_DATASET = 'AffectiveROAD'
    data_group = np.load(f'{DATA_DIR}/{NAME_DATASET}_groups_1.npy')
    data_gt = np.load(f'{DATA_DIR}/{NAME_DATASET}_ground_truth_1.npy')
    data_ft = np.load(f'{DATA_DIR}/{NAME_DATASET}_stats_feats_1.npy')
    indices = np.where(data_gt >= 0)[0]
    data_ft = data_ft[indices]
    data_group = data_group[indices]
    data_gt = data_gt[indices]

# Create dataframe for dataset
column_values = [f'f{x}' for x in range(data_ft.shape[1])]
data_full = pd.DataFrame(data = data_ft,  
                         columns = column_values)
data_full['subject_id'] = data_group
data_full['label'] = data_gt
list_subject_id = np.unique(data_full['subject_id']).tolist()
ft_names = data_full.columns.tolist()
##### TRAIN / VAL / TEST #####
print(f'Processing {subject_id_test}')
data_test = data_full[data_full.subject_id == subject_id_test]

data_train_val = data_full[data_full.subject_id != subject_id_test]
print(len(data_test))
list_id = list(set(data_train_val.subject_id))
list_id.sort()

Processing S14
2721


In [2]:
# Model
class Embedding_Model(nn.Module):
    def __init__(self, input_size, dropout=0.4, emb_size=[64], activation='relu'):
        super(Embedding_Model, self).__init__()
        self.do = dropout
        self.input_size = input_size
        self.emb_size = emb_size
        
        modules = [] 
        for idx, size in enumerate(self.emb_size):
            if idx != len(self.emb_size) - 1: # Not the last layer
                if idx == 0:
                    modules.append(nn.Linear(self.input_size, self.emb_size[idx]))
                else:
                    modules.append(nn.Linear(self.emb_size[idx-1], self.emb_size[idx]))
                modules.append(nn.BatchNorm1d(num_features=self.emb_size[idx]))
                if activation == 'relu':
                    modules.append(nn.ReLU())
                else:
                    modules.append(MemoryEfficientSwish())
                modules.append(nn.Dropout(self.do))   
            else:
                # This is the last layer
                if idx == 0:
                    modules.append(nn.Linear(self.input_size, self.emb_size[idx]))
                else:
                    modules.append(nn.Linear(self.emb_size[idx-1], self.emb_size[idx]))
 
        self.emb = nn.Sequential(*modules)
        
    def forward(self, feat):
        x = self.emb(feat)
        #x = F.normalize(x, p=2, dim=1)
        return x
    
class Model_CLS(nn.Module):
    def __init__(self, input_eda_size, input_hr_size, emb_eda_size=None, emb_hr_size=None, 
                 dropout=0.2, cls_size=None, activation='relu', use_res=False):
        # cls_size, emb_eda_size, emb_hr_size is None or a list
        
        super(Model_CLS, self).__init__()
        self.dropout = dropout
        
        if emb_eda_size[-1] != input_eda_size or emb_hr_size[-1] != input_hr_size:
            self.use_res = False
        else:
            self.use_res = use_res
            
        self.input_eda_size = input_eda_size
        if emb_eda_size is None:
            self.emb_eda_size = [input_eda_size]
        else:
            self.emb_eda_size = emb_eda_size
        self.eda_encode = Embedding_Model(input_size=self.input_eda_size, 
                                          dropout=self.dropout, 
                                          emb_size=self.emb_eda_size, activation=activation)
        
        self.input_hr_size = input_hr_size
        if emb_hr_size is None:
            self.emb_hr_size = [input_hr_size]
        else:
            self.emb_hr_size = emb_hr_size
        self.hr_encode = Embedding_Model(input_size=self.input_hr_size, 
                                         dropout=self.dropout, 
                                         emb_size=self.emb_hr_size, activation=activation)
        
        self.input_cls_size = self.emb_eda_size[-1] + self.emb_hr_size[-1]
        if cls_size is None:
            self.cls_size = [1]
        else:
            self.cls_size = cls_size + [1]
        self.cls = Embedding_Model(input_size=self.input_cls_size, 
                                   dropout=self.dropout, 
                                   emb_size=self.cls_size, activation=activation)
        
        self.eda_cls = Embedding_Model(input_size=self.emb_eda_size[-1], 
                                       dropout=self.dropout, 
                                       emb_size=[1], activation=activation)
        self.hr_cls = Embedding_Model(input_size=self.emb_hr_size[-1], 
                                       dropout=self.dropout, 
                                       emb_size=[1], activation=activation)
            
    def forward(self, eda_ft, hr_ft):
        eda_emb = self.eda_encode(eda_ft)
        hr_emb = self.hr_encode(hr_ft)
        
        if self.use_res:
            eda_emb = eda_emb + eda_ft
            hr_emb = hr_emb + hr_ft
            
        eda_logit = self.eda_cls(eda_emb)
        hr_logit = self.hr_cls(hr_emb)
        
        com_emb = torch.cat((eda_emb,hr_emb), dim=1)
        #if self.use_res:
        #    com_emb = com_emb + torch.cat((eda_ft, hr_ft), dim=1)
        com_logit = self.cls(com_emb)
        
        return eda_logit, hr_logit, com_logit

In [3]:
# Dataset
class EmbDataset(Dataset):
    def __init__(self, df):
        '''
        df is the dataframe containing features + subject_id + label
        numb_samples (int) total of pairs (samples) want to generated
        '''
        self.df = df.copy()
        
    def __getitem__(self, i):
        sample_1 = self.df.iloc[i,:-2].to_numpy(dtype=np.float64) # not include subject_id and label
        label_1 = self.df['label'][i]
        hr_ft = sample_1[0:25]
        eda_ft = sample_1[25:]
        return eda_ft, hr_ft, label_1
        
    def __len__(self):
        return len(self.df)
    
def generate_batch_embed(batch):
    eda_fts, hr_fts,labels = zip(*batch)
    eda_ft = torch.tensor([ft for ft in eda_fts]).squeeze(1).float()#.to(device)
    hr_ft = torch.tensor([ft for ft in hr_fts]).squeeze(1).float()
    labels = torch.tensor(labels).float()#.to(device)
    return eda_ft, hr_ft, labels

# Dataloader Class
def make_EmbDataLoader(dataset, **args):
    data = DataLoader(dataset, collate_fn=generate_batch_embed, **args)
    return data

In [4]:
def test_model_dict(model_dict, dataloader_dict):
    list_validate_id = list(model_dict.keys())
    all_pred_eda_dict = {}
    all_pred_hr_dict = {}
    all_pred_com_dict = {}
    all_pred_cat_dict = {}
    all_pred_eda_prob_dict = {}
    all_pred_hr_prob_dict = {}
    all_pred_com_prob_dict = {}
    all_pred_cat_prob_dict = {}
    
    with torch.no_grad():
        for idx_id, subject_id_validate in enumerate(list_validate_id):
            dataloader = dataloader_dict[subject_id_validate]
            model = model_dict[subject_id_validate]
            model.eval()
            
            all_pred_eda = []
            all_pred_hr = []
            all_pred_com = []
            all_pred_cat = []
            all_pred_eda_prob = []
            all_pred_hr_prob = []
            all_pred_com_prob = []
            all_pred_cat_prob = []
    
            if idx_id == 0:
                all_label = []
                
            for i, (eda_ft, hr_ft, labels) in enumerate(dataloader):
                eda_ft = eda_ft.to(device)
                hr_ft = hr_ft.to(device)
                labels = labels.to(device)
                labels = labels.unsqueeze(-1)

                eda_logit, hr_logit, com_logit = model(eda_ft, hr_ft)
                eda_cls = sigmoid_function(eda_logit)
                hr_cls = sigmoid_function(hr_logit)
                com_cls = sigmoid_function(com_logit)
                cat_cls = (eda_cls + hr_cls + com_cls)/3
            
                preds_eda_round_list = eda_cls.round().squeeze().detach().cpu().numpy().tolist()
                preds_hr_round_list = hr_cls.round().squeeze().detach().cpu().numpy().tolist()
                preds_com_round_list = com_cls.round().squeeze().detach().cpu().numpy().tolist()
                preds_cat_round_list = cat_cls.round().squeeze().detach().cpu().numpy().tolist()
                
                preds_eda_prob_list = eda_cls.squeeze().detach().cpu().numpy().tolist()
                preds_hr_prob_list = hr_cls.squeeze().detach().cpu().numpy().tolist()
                preds_com_prob_list = com_cls.squeeze().detach().cpu().numpy().tolist()
                preds_cat_prob_list = cat_cls.squeeze().detach().cpu().numpy().tolist()
                
                labels = labels.round().squeeze().detach().cpu().numpy().tolist()
                
                all_pred_eda += preds_eda_round_list
                all_pred_com += preds_com_round_list
                all_pred_hr += preds_hr_round_list
                all_pred_cat += preds_cat_round_list
                all_pred_eda_prob += preds_eda_prob_list
                all_pred_hr_prob += preds_hr_prob_list
                all_pred_com_prob += preds_com_prob_list
                all_pred_cat_prob += preds_cat_prob_list
                
                if idx_id == 0:
                    all_label += labels
                    
            all_pred_eda_dict[subject_id_validate] = all_pred_eda
            all_pred_hr_dict[subject_id_validate] = all_pred_hr
            all_pred_com_dict[subject_id_validate] = all_pred_com
            all_pred_cat_dict[subject_id_validate] = all_pred_cat
            all_pred_eda_prob_dict[subject_id_validate] = all_pred_eda_prob
            all_pred_hr_prob_dict[subject_id_validate] = all_pred_hr_prob
            all_pred_com_prob_dict[subject_id_validate] = all_pred_com_prob
            all_pred_cat_prob_dict[subject_id_validate] = all_pred_cat_prob
    
    for idx, subject_id_validate in enumerate(list_validate_id):
        if idx == 0:
            f_pred_eda = np.array([all_pred_eda_dict[subject_id_validate]])
            f_pred_hr = np.array([all_pred_hr_dict[subject_id_validate]])
            f_pred_com = np.array([all_pred_com_dict[subject_id_validate]])
            f_pred_cat = np.array([all_pred_cat_dict[subject_id_validate]])
            f_pred_eda_prob = np.array([all_pred_eda_prob_dict[subject_id_validate]])
            f_pred_hr_prob = np.array([all_pred_hr_prob_dict[subject_id_validate]])
            f_pred_com_prob = np.array([all_pred_com_prob_dict[subject_id_validate]])
            f_pred_cat_prob = np.array([all_pred_cat_prob_dict[subject_id_validate]])
        else:
            f_pred_eda = np.concatenate((f_pred_eda, np.array([all_pred_eda_dict[subject_id_validate]])), axis=0)
            f_pred_hr = np.concatenate((f_pred_hr, np.array([all_pred_hr_dict[subject_id_validate]])), axis=0)
            f_pred_com = np.concatenate((f_pred_com, np.array([all_pred_com_dict[subject_id_validate]])), axis=0)
            f_pred_cat = np.concatenate((f_pred_cat, np.array([all_pred_cat_dict[subject_id_validate]])), axis=0)
            f_pred_eda_prob = np.concatenate((f_pred_eda_prob, np.array([all_pred_eda_prob_dict[subject_id_validate]])), axis=0)
            f_pred_hr_prob = np.concatenate((f_pred_hr_prob, np.array([all_pred_hr_prob_dict[subject_id_validate]])), axis=0)
            f_pred_com_prob = np.concatenate((f_pred_com_prob, np.array([all_pred_com_prob_dict[subject_id_validate]])), axis=0)
            f_pred_cat_prob = np.concatenate((f_pred_cat_prob, np.array([all_pred_cat_prob_dict[subject_id_validate]])), axis=0)
    
    
    f_pred_eda = np.mean(f_pred_eda, axis=0)
    f_pred_hr = np.mean(f_pred_hr, axis=0)
    f_pred_com = np.mean(f_pred_com, axis=0)
    f_pred_cat = np.mean(f_pred_cat, axis=0)
    f_pred_eda_prob = np.mean(f_pred_eda_prob, axis=0)
    f_pred_hr_prob = np.mean(f_pred_hr_prob, axis=0)
    f_pred_com_prob = np.mean(f_pred_com_prob, axis=0)
    f_pred_cat_prob = np.mean(f_pred_cat_prob, axis=0)
    
    f_pred_eda = np.round(f_pred_eda)
    f_pred_hr = np.round(f_pred_hr)
    f_pred_com = np.round(f_pred_com)
    f_pred_cat = np.round(f_pred_cat)
    f_pred_eda_prob = np.round(f_pred_eda_prob)
    f_pred_hr_prob = np.round(f_pred_hr_prob)
    f_pred_com_prob = np.round(f_pred_com_prob)
    f_pred_cat_prob = np.round(f_pred_cat_prob)
    
    bacc_eda = balanced_accuracy_score(all_label, f_pred_eda)
    acc_eda = accuracy_score(all_label, f_pred_eda)
    f1_eda = f1_score(all_label, f_pred_eda)    
    
    bacc_hr = balanced_accuracy_score(all_label, f_pred_hr)
    acc_hr = accuracy_score(all_label, f_pred_hr)
    f1_hr = f1_score(all_label, f_pred_hr)  
    
    bacc_com = balanced_accuracy_score(all_label, f_pred_com)
    acc_com = accuracy_score(all_label, f_pred_com)
    f1_com = f1_score(all_label, f_pred_com) 
    
    bacc_cat = balanced_accuracy_score(all_label, f_pred_cat)
    acc_cat = accuracy_score(all_label, f_pred_cat)
    f1_cat = f1_score(all_label, f_pred_cat) 
    
    bacc_eda_prob = balanced_accuracy_score(all_label, f_pred_eda_prob)
    acc_eda_prob = accuracy_score(all_label, f_pred_eda_prob)
    f1_eda_prob = f1_score(all_label, f_pred_eda_prob)    
    
    bacc_hr_prob = balanced_accuracy_score(all_label, f_pred_hr_prob)
    acc_hr_prob = accuracy_score(all_label, f_pred_hr_prob)
    f1_hr_prob = f1_score(all_label, f_pred_hr_prob)  
    
    bacc_com_prob = balanced_accuracy_score(all_label, f_pred_com_prob)
    acc_com_prob = accuracy_score(all_label, f_pred_com_prob)
    f1_com_prob = f1_score(all_label, f_pred_com_prob) 
    
    bacc_cat_prob = balanced_accuracy_score(all_label, f_pred_cat_prob)
    acc_cat_prob = accuracy_score(all_label, f_pred_cat_prob)
    f1_cat_prob = f1_score(all_label, f_pred_cat_prob) 
    
    metric_dict = {}
    metric_dict['eda'] = {}
    metric_dict['eda']['acc'] = acc_eda
    metric_dict['eda']['bacc'] = bacc_eda
    metric_dict['eda']['f1'] = f1_eda
    metric_dict['hr'] = {}
    metric_dict['hr']['acc'] = acc_hr
    metric_dict['hr']['bacc'] = bacc_hr
    metric_dict['hr']['f1'] = f1_hr
    metric_dict['com'] = {}
    metric_dict['com']['acc'] = acc_com
    metric_dict['com']['bacc'] = bacc_com
    metric_dict['com']['f1'] = f1_com
    metric_dict['cat'] = {}
    metric_dict['cat']['acc'] = acc_cat
    metric_dict['cat']['bacc'] = bacc_cat
    metric_dict['cat']['f1'] = f1_cat
    metric_dict['eda_prob'] = {}
    metric_dict['eda_prob']['acc'] = acc_eda_prob
    metric_dict['eda_prob']['bacc'] = bacc_eda_prob
    metric_dict['eda_prob']['f1'] = f1_eda_prob
    metric_dict['hr_prob'] = {}
    metric_dict['hr_prob']['acc'] = acc_hr_prob
    metric_dict['hr_prob']['bacc'] = bacc_hr_prob
    metric_dict['hr_prob']['f1'] = f1_hr_prob
    metric_dict['com_prob'] = {}
    metric_dict['com_prob']['acc'] = acc_com_prob
    metric_dict['com_prob']['bacc'] = bacc_com_prob
    metric_dict['com_prob']['f1'] = f1_com_prob
    metric_dict['cat_prob'] = {}
    metric_dict['cat_prob']['acc'] = acc_cat_prob
    metric_dict['cat_prob']['bacc'] = bacc_cat_prob
    metric_dict['cat_prob']['f1'] = f1_cat_prob
    
    return metric_dict

In [5]:
#Test
print('Run on Test set in final ...')
X_test = data_test.iloc[:,:-2].to_numpy()
group_test = data_test.iloc[:,-2].to_numpy()
y_test = data_test.iloc[:,-1].to_numpy()
model_dict = {}
dataloader_dict = {}
for subject_id_validate in list_id:
    # Scaler Data
    scaler = joblib.load(f'{SAVE_MODEL_DIR}/{NAME_DATASET}/Model/{SUBFOLDER}/StandardScaler_{subject_id_test}_{subject_id_validate}.joblib')
    X_test_scaler = scaler.transform(X_test)
    # Create Dataframe
    df_test= pd.DataFrame(data = X_test_scaler, columns = ft_names[:-2])
    df_test['subject_id'] = group_test
    df_test['label'] = y_test
    # Create Dataset and Dataloader
    test_dataset = EmbDataset(df_test)
    test_dataloader = make_EmbDataLoader(test_dataset, batch_size=2048, shuffle=False)
    dataloader_dict[subject_id_validate] = test_dataloader
    # Load Model
    Model = Model_CLS(input_eda_size=EDA_FT, input_hr_size=HR_FT, 
                      emb_eda_size=EMB_EDA_SIZE, emb_hr_size=EMB_HR_SIZE, 
                      dropout=DROPOUT, cls_size=CLS_SIZE, activation=ACTIVATION, use_res=USE_RES)
    Model = Model.to(device)

    modelCheckpoint = torch.load(f"{SAVE_MODEL_DIR}/{NAME_DATASET}/Model/{SUBFOLDER}/{MODEL_NAME}_{subject_id_test}_{subject_id_validate}.pth.tar")
    Model.load_state_dict(modelCheckpoint['model_state_dict'])
    model_dict[subject_id_validate] = Model
    Model.eval()
    # Predict
    #test_metric_dict = validate_epoch(Model, test_dataloader)
    #test_metric_info = metric_to_info(test_metric_dict)
    #test_info = f'[CLASSIFY TEST {subject_id_test} - VALIDATE {subject_id_validate}]\n'
    #test_info += test_metric_info
    #print(test_info)
    #logging(test_info + '\n', f'{SAVE_MODEL_DIR}/{NAME_DATASET}/Log/{SUBFOLDER}/Test_{MODEL_NAME}_{subject_id_test}.txt', True)

validate_test_metric_dict = test_model_dict(model_dict, dataloader_dict)
#test_metric_info = metric_to_info(validate_test_metric_dict)
#test_info = f'[FINAL CLASSIFY TEST {subject_id_test}]\n'
#test_info += test_metric_info
#print(test_info)
#logging(test_info + '\n', f'{SAVE_MODEL_DIR}/{NAME_DATASET}/Log/{SUBFOLDER}/Test_{MODEL_NAME}_{subject_id_test}.txt', True)

Run on Test set in final ...


In [6]:
validate_test_metric_dict

{'eda': {'acc': 0.7894156560088202,
  'bacc': 0.5341463414634147,
  'f1': 0.1278538812785388},
 'hr': {'acc': 0.8397647923557515,
  'bacc': 0.7951821740439626,
  'f1': 0.6681887366818874},
 'com': {'acc': 0.8213891951488423,
  'bacc': 0.6048780487804878,
  'f1': 0.34677419354838707},
 'cat': {'acc': 0.8213891951488423,
  'bacc': 0.6048780487804878,
  'f1': 0.34677419354838707},
 'eda_prob': {'acc': 0.788680632120544,
  'bacc': 0.532520325203252,
  'f1': 0.1221374045801527},
 'hr_prob': {'acc': 0.8401323042998897,
  'bacc': 0.7971463646260395,
  'f1': 0.670204700530705},
 'com_prob': {'acc': 0.8195516354281515,
  'bacc': 0.6008130081300813,
  'f1': 0.33558863328822736},
 'cat_prob': {'acc': 0.8195516354281515,
  'bacc': 0.6008130081300813,
  'f1': 0.33558863328822736}}